# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

In [2]:
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

pd.set_option("display.max_rows", None, "display.max_columns", None)

**Zadanie 1** Znajdź listę wszystkich filmów o tej samej długości.

In [125]:
df = pd.read_sql('\
SELECT length, COUNT(film_id) "number of films" FROM film \
GROUP BY length \
ORDER BY length ASC \
',con=connection)
print(df)

     length  number of films
0        46                5
1        47                7
2        48               11
3        49                5
4        50                9
5        51                7
6        52                7
7        53                9
8        54                6
9        55                2
10       56                5
11       57                7
12       58                7
13       59                9
14       60                8
15       61               10
16       62                6
17       63                9
18       64                9
19       65                7
20       66                2
21       67                8
22       68                5
23       69                6
24       70                7
25       71                7
26       72                4
27       73               12
28       74               12
29       75               10
30       76                7
31       77                6
32       78                6
33       79   

/home/kuba/anaconda3/envs/jakubbajek/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


**Zadanie 2** Znajdź wszystkich klientów mieszkających w tym samym mieście.

In [141]:
df = pd.read_sql('\
SELECT c.city, COUNT(cu.customer_id) "customers" FROM customer cu \
INNER JOIN address a ON a.address_id = cu.address_id \
INNER JOIN city c ON c.city_id = a.city_id \
GROUP BY c.city \
ORDER BY c.city \
',con=connection)
print(df)

                           city  customers
0            A Corua (La Corua)          1
1                          Abha          1
2                     Abu Dhabi          1
3                          Acua          1
4                         Adana          1
5                   Addis Abeba          1
6                          Aden          1
7                         Adoni          1
8                    Ahmadnagar          1
9                      Akishima          1
10                        Akron          1
11                       al-Ayn          1
12                  Alessandria          1
13                    al-Hawiya          1
14        Allappuzha (Alleppey)          1
15                      Allende          1
16                    al-Manama          1
17              Almirante Brown          1
18                   al-Qadarif          1
19                     al-Qatif          1
20                     Alvorada          1
21                     Ambattur          1
22         

/home/kuba/anaconda3/envs/jakubbajek/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


**Zadanie 3** Oblicz średni koszt wypożyczenia wszystkich filmów.

In [160]:
# Średnia zapłata przy pożyczaniu danego filmu
# df = pd.read_sql('\
# SELECT f.film_id, f.title, AVG(p.amount) "average rental cost" FROM rental r \
# INNER JOIN inventory i ON i.inventory_id = r.inventory_id \
# INNER JOIN film f ON f.film_id = i.film_id \
# INNER JOIN payment p ON p.rental_id = r.rental_id \
# GROUP BY f.film_id \
# ORDER BY f.film_id \
# ',con=connection)
# print(df)

df = pd.read_sql('\
SELECT AVG(amount) "Average cost of film rental" FROM payment \
',con=connection)
print(df)

   Average cost of film rental
0                     4.200606


/home/kuba/anaconda3/envs/jakubbajek/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


**Zadanie 4** Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.

In [164]:
df = pd.read_sql('\
SELECT c.name category, COUNT(f.film_id) "number of films" FROM film f \
INNER JOIN film_category fc ON fc.film_id = f.film_id \
INNER JOIN category c ON c.category_id = fc.category_id \
GROUP BY category \
ORDER BY category ASC\
',con=connection)
print(df)

       category  number of films
0        Action               64
1     Animation               66
2      Children               60
3      Classics               57
4        Comedy               58
5   Documentary               68
6         Drama               62
7        Family               69
8       Foreign               73
9         Games               61
10       Horror               56
11        Music               51
12          New               63
13       Sci-Fi               61
14       Sports               74
15       Travel               57


/home/kuba/anaconda3/envs/jakubbajek/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


**Zadanie 5** Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.

In [174]:
df = pd.read_sql('\
SELECT co.country, COUNT(cu.customer_id) "customers" FROM customer cu \
INNER JOIN address a ON a.address_id = cu.address_id \
INNER JOIN city ci ON ci.city_id = a.city_id \
INNER JOIN country co ON co.country_id = ci.country_id \
GROUP BY co.country \
ORDER BY customers DESC, co.country\
',con=connection)
print(df)

                                   country  customers
0                                    India         60
1                                    China         53
2                            United States         36
3                                    Japan         31
4                                   Mexico         30
5                                   Brazil         28
6                       Russian Federation         28
7                              Philippines         20
8                                   Turkey         15
9                                Indonesia         14
10                               Argentina         13
11                                 Nigeria         13
12                            South Africa         11
13                                  Taiwan         10
14                          United Kingdom          9
15                                    Iran          8
16                                  Poland          8
17                          

/home/kuba/anaconda3/envs/jakubbajek/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


**Zadanie 6** Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.

In [197]:
df = pd.read_sql('\
SELECT store.store_id ,COUNT(customer.customer_id) customers, \
staff.first_name "manager first name", staff.last_name "manager last name", \
address.address "store address", city.city, country.country FROM store \
INNER JOIN customer ON customer.store_id = store.store_id \
INNER JOIN staff ON staff.staff_id = store.manager_staff_id \
INNER JOIN address ON address.address_id = store.address_id \
INNER JOIN city ON address.city_id = city.city_id \
INNER JOIN country ON country.country_id = city.country_id \
GROUP BY store.store_id, staff.first_name, staff.last_name, address.address, city.city, country.country \
HAVING COUNT(customer.customer_id) > 100 AND COUNT(customer.customer_id) < 300\
',con=connection)
df

,store_id,customers,manager first name,manager last name,store address,city,country
0,2,273,Jon,Stephens,28 MySQL Boulevard,Woodridge,Australia


**Zadanie 7** Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.

In [18]:
#Nie używam zagnieżdzonego SELECTa, żeby wyświetlić też czas oglądania
df = pd.read_sql('\
SELECT c.customer_id, c.first_name, c.last_name, SUM(f.length) "watching time (minutes)" FROM customer c \
INNER JOIN rental r ON r.customer_id = c.customer_id \
INNER JOIN inventory i ON i.inventory_id = r.inventory_id \
INNER JOIN film f ON f.film_id = i.film_id \
GROUP BY c.customer_id \
HAVING SUM(f.length) > 60*200 \
ORDER BY c.customer_id \
',con=connection)
print('Klienci ogladajacy ponad 200 godzin:\n',df)

df = pd.read_sql('\
SELECT c.customer_id, c.first_name, c.last_name, SUM(f.length)/60 "watching time (hours)" FROM customer c \
INNER JOIN rental r ON r.customer_id = c.customer_id \
INNER JOIN inventory i ON i.inventory_id = r.inventory_id \
INNER JOIN film f ON f.film_id = i.film_id \
GROUP BY c.customer_id \
HAVING SUM(f.length)/60 > 75 \
ORDER BY "watching time (hours)", c.customer_id \
',con=connection)
print('\nKlienci ogladajacy ponad 75 godzin:\n',df)

Klienci ogladajacy ponad 200 godzin:
 Empty DataFrame
Columns: [customer_id, first_name, last_name, watching time (minutes)]
Index: []

Klienci ogladajacy ponad 75 godzin:
    customer_id first_name last_name  watching time (hours)
0          119     Sherry  Marshall                     76
1          178     Marion    Snyder                     76
2          380    Russell   Brinson                     77
3          236     Marcia      Dean                     78
4          144      Clara      Shaw                     80
5          469     Wesley      Bull                     80
6           75      Tammy   Sanders                     84
7          148    Eleanor      Hunt                     89
8          526       Karl      Seal                     89


/home/kuba/anaconda3/envs/jakubbajek/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/kuba/anaconda3/envs/jakubbajek/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


**Zadanie 8** Oblicz średnią wartość wypożyczenia filmu.<br>
W mainie było pytanie o średnią wartość dla zadanej długości filmów, z przykładowej linijki wywnioskowałem, że chodzi o średnią kwotę jaką trzeba zapłacić za wypożyczenie filmów o tej długości za jeden film.<br>
Tutaj jest pytanie o obliczenie dla wszystkich filmów, co pokryłoby się z zadaniem 3, dlatego w zadaniu obliczę średni rental_rate dla wszystkich filmów, wydaje mi się to najbardziej odpowiednie do pytania

In [235]:
df = pd.read_sql('\
SELECT AVG(rental_rate) "Average rental rate" FROM film\
',con=connection)
print(df)

   Average rental rate
0                 2.98


**Zadanie 9** Oblicz średnią wartość długości filmu we wszystkich kategoriach.

In [241]:
df = pd.read_sql('\
SELECT c.name category, AVG(f.length) "avg length (min)" from film f \
INNER JOIN film_category fc ON fc.film_id = f.film_id \
INNER JOIN category c ON c.category_id = fc.category_id \
GROUP BY category \
ORDER BY category \
',con=connection)
print(df)

       category  avg length (min)
0        Action        111.609375
1     Animation        111.015152
2      Children        109.800000
3      Classics        111.666667
4        Comedy        115.827586
5   Documentary        108.750000
6         Drama        120.838710
7        Family        114.782609
8       Foreign        121.698630
9         Games        127.836066
10       Horror        112.482143
11        Music        113.647059
12          New        111.126984
13       Sci-Fi        108.196721
14       Sports        128.202703
15       Travel        113.315789


/home/kuba/anaconda3/envs/jakubbajek/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


**Zadanie 10** Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach. <br>
W niektórych kategoriach więcej niż jeden tytuł jest najdłuższy <br>
W zadaniu wykorzystałem zagnieżdżone zapytanie SELECT do stworzenia tablicy z pogrupowaną według *category_id* największą długością tytułu (max liczba znaków) <br>
Następnie przyłączyłem do niej pasujący tytuł na podstawie *category_id* i tej największej ilości znaków

In [40]:
df = pd.read_sql('\
SELECT c.name category, f.title "longest title" FROM(\
    SELECT fc.category_id cid, MAX(LENGTH(f.title)) max from film f \
    INNER JOIN film_category fc ON fc.film_id = f.film_id \
    GROUP BY cid \
    ) AS subquery \
INNER JOIN category c ON c.category_id = subquery.cid \
INNER JOIN film_category fc ON fc.category_id = c.category_id \
INNER JOIN film f ON f.film_id = fc.film_id AND LENGTH(f.title) = subquery.max \
ORDER BY category, "longest title"\
',con=connection)
print(df)

       category                longest title
0        Action      Entrapment Satisfaction
1     Animation       Telemark Heartbreakers
2      Children         Heartbreakers Bright
3      Children         Microcosmos Paradise
4      Children         Sweethearts Suspects
5      Classics      Extraordinary Conquerer
6        Comedy      Trainspotting Strangers
7   Documentary       Deliverance Mulholland
8   Documentary       Intolerable Intentions
9         Drama       Goldfinger Sensibility
10       Family       Resurrection Silverado
11      Foreign         Ballroom Mockingbird
12      Foreign         Impossible Prejudice
13      Foreign         Reservoir Adaptation
14        Games        Bulworth Commandments
15        Games        Creatures Shakespeare
16       Horror  Arachnophobia Rollercoaster
17        Music       Confidential Interview
18          New        Flamingos Connecticut
19       Sci-Fi         Barbarella Streetcar
20       Sci-Fi         Cincinatti Whisperer
21       S

/home/kuba/anaconda3/envs/jakubbajek/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


**Zadanie 11** Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10. <br> Wyniki różnią się tak jak powinny. Nauczyło mnie to, że MAX(string) zwraca nie najdłuższą wartość, a alfabetycznie ostatnią

In [55]:
df = pd.read_sql('\
SELECT c.name category, f.title "longest film", subquery.max "length (min)", subquery.max / CAST(60 AS float) "length (hours)" FROM(\
    SELECT fc.category_id cid, MAX(f.length) max from film f \
    INNER JOIN film_category fc ON fc.film_id = f.film_id \
    GROUP BY cid \
    ) AS subquery \
INNER JOIN category c ON c.category_id = subquery.cid \
INNER JOIN film_category fc ON fc.category_id = c.category_id \
INNER JOIN film f ON f.film_id = fc.film_id AND f.length = subquery.max \
ORDER BY category, "longest film"\
',con=connection)
print(df)

       category        longest film  length (min)  length (hours)
0        Action      Darn Forrester           185        3.083333
1        Action        Worst Banger           185        3.083333
2     Animation         Gangs Pride           185        3.083333
3     Animation        Pond Seattle           185        3.083333
4      Children         Fury Murder           178        2.966667
5      Children      Wrong Behavior           178        2.966667
6      Classics   Conspiracy Spirit           184        3.066667
7        Comedy      Control Anthem           185        3.083333
8   Documentary           Wife Turn           183        3.050000
9   Documentary      Young Language           183        3.050000
10        Drama       Jacket Frisco           181        3.016667
11       Family      King Evolution           184        3.066667
12      Foreign    Crystal Breaking           184        3.066667
13      Foreign      Sorority Queen           184        3.066667
14        

## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [53]:
import main

In [59]:
print(main.film_in_category(3))

                   title               languge  category
0    Backlash Undefeated  English               Children
1         Bear Graceland  English               Children
2           Beneath Rush  English               Children
3          Betrayed Rear  English               Children
4            Cabin Flash  English               Children
5       Casper Dragonfly  English               Children
6    Christmas Moonshine  English               Children
7           Circus Youth  English               Children
8     Clockwork Paradise  English               Children
9      Comancheros Enemy  English               Children
10       Crooked Frogmen  English               Children
11      Daughter Madigan  English               Children
12          Doctor Grail  English               Children
13      Empire Malkovich  English               Children
14          Fargo Gandhi  English               Children
15     Forever Candidate  English               Children
16       Full Flatliners  Engli

/home/kuba/anaconda3/envs/jakubbajek/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [60]:
print(main.number_films_in_category(3))

   category  count
0  Children     60


/home/kuba/anaconda3/envs/jakubbajek/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [67]:
print(main.number_film_by_length(89,106))

    length  count
0       89      6
1       90      5
2       91      6
3       92     11
4       93      8
5       94      4
6       95      2
7       96      2
8       97      4
9       98      8
10      99      8
11     100     12
12     101      7
13     102     11
14     103      9
15     104      6
16     105      6
17     106      6


/home/kuba/anaconda3/envs/jakubbajek/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [72]:
print(main.client_from_city('York'))
print()
print(main.client_from_city('Athenai'))

   city first_name last_name
0  York    Gilbert    Sledge

      city first_name last_name
0  Athenai      Linda  Williams


/home/kuba/anaconda3/envs/jakubbajek/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/kuba/anaconda3/envs/jakubbajek/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [75]:
print(main.avg_amount_by_length(48))

   length       avg
0      48  4.295389


/home/kuba/anaconda3/envs/jakubbajek/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [76]:
print(main.client_by_sum_length(1265))

      first_name     last_name   sum
0          Brian         Wyman  1265
1        Antonio          Meek  1451
2          Leona        Obrien  1588
3      Katherine        Rivera  1615
4        Tiffany        Jordan  1667
5         Jerome        Kenyon  1679
6          Penny          Neal  1738
7         Lonnie        Tirado  1834
8         Dwight      Lombardi  1846
9         Johnny        Turpin  1846
10      Caroline        Bowman  1847
11         Allan       Cornish  1916
12         Lewis         Lyman  1933
13        Lester         Kraus  1943
14           Eva         Ramos  1945
15        Austin       Cintron  1961
16         Joann       Gardner  1963
17         Annie       Russell  1965
18      Jonathan   Scarborough  1984
19         Anita       Morales  1999
20        Eileen          Carr  2031
21       Kenneth        Gooden  2033
22         Wayne        Truong  2044
23         Floyd         Gandy  2063
24         Paula        Bryant  2084
25        Nellie       Garrett  2098
2

/home/kuba/anaconda3/envs/jakubbajek/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [78]:
print(main.category_statistic_length('Action'))
print()
print(main.category_statistic_length('Children'))

  category         avg   sum  min  max
0   Action  111.609375  7143   47  185

   category    avg   sum  min  max
0  Children  109.8  6588   46  178


/home/kuba/anaconda3/envs/jakubbajek/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/kuba/anaconda3/envs/jakubbajek/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
